<h1> Statistical Analysis

In [ ]:
## Numpy
import numpy as np

#? Creating array
#* Creating 4x2 numpy array
n1 = np.array([[1,2],[3,4],[5,6],[7,8]], dtype=np.int64)
# dtype = np.int64, np.float64

#* Creating 2x3 zero array
n1 = np.zeros((2,3))

#* Creating 1x4 one array
n1 = np.ones((1,4))

#* Creating 5x5 constant array
n1 = np.full((5,5), 100)                    # Only takes integer

#* Creating array of same element 
n1 = np.repeat([0.5, 0.25], 3, axis=0)      # Takes float too

#* Creating 2x2 random array
n1 = np.random.random((2,2))

#* Creating 1xn increasing array
n1 = np.arange(start=0, stop=11, step=1, dtype=int)
np.arange('2020-04-01', '2024-07-15', 3, dtype='datetime64[M]').astype('datetime64[D]')                             # Create quarterly datetime
np.arange(np.datetime64('2022-04-01'), np.datetime64('2023-05-01'), np.timedelta64(1, 'M'), dtype='datetime64[M]')  # Create monthly datetime

#* Return evenly spaced array
n1 = np.linspace(0, 10, num=10)

#* Append values to the end of array
n1 = np.append([1,2,3],[4,5,6], axis=1)


#? Array shape
#* Check shape of array
n1.shape

#* Reshape array
n1.reshape((5,1))

#* Return an array of ones/bool with same shape
np.ones_like(n1)
np.ones_like(n1, dtype=bool)    # Return bool instead of 1

#* Return upper triangle of array
np.triu(n1)


#? Slicing array
#* Return individual value
x = n1[2,3]

#* Return subarray
n2 = n1[:2, 1:3]

#* Replace value
n1[0,0] = 100


#? Math
#* Add arrays together
n3 = n1 + n2

#* Sum up array
n3 = np.sum(n1)

#* Sum up array by column/row
n3 = np.sum(n1, axis=0)
# column: axis=0, row: axis=1

#* Average array
n3 = np.mean(n1)
n3 = np.mean(n1, axis=0)





In [ ]:
## Generate random variables
import pandas as pd
from scipy import stats
import random

random.seed(1234)       # for random
np.random.seed(1234)    # for numpy and scipy


#? Generate univariate random variables
#* Normal variables
random.normalvariate(mu=0, sigma=1)
np.random.normal(loc=0, scale=1, size=100)
stats.norm.rvs(loc=0, scale=1, size=100)

#* t variables
np.random.standard_t(10, size=100)
stats.t.rvs(10, size=100)

#* Uniform variables
np.random.uniform(low=0, high=1, size=100)

#* Discrete variables
np.random.randint(low=10, high=100, size=(4,4))


#? Generate multivariate random variables
#* Normal variables
np.random.multivariate_normal(mean=[0,0], cov=[[1,0], [0,100]], size=100)


#? Random sampling
np.random.choice(10,3,replace=True)     # from np.arange(10) choose 3 with replacement
np.random.choice([.1,.3,.5,.7],3)       # choose from list without replacement


#? Generate sampling distribution (mean)
#* Simulated
popu = np.random.normal(loc=65, scale=15, size=100000)  # start with known list of N population units
samp = np.random.choice(popu, (25000,500))              # randomly select n units from list, multiple times
samp_mean = samp.mean(axis=1)                           # calculate sample means
# population size, N = 100000
# sample size, n = 500
# number of samples taken = 25000
# every population unit has equal probability of selection = n/N
# estimates of parameters based on SRS are UNBIASED (equal to population values on average)

#* Subsampling
df1 = pd.read_csv('probability_sampled_data.csv')
for m in 100, 200, 400, 800:     # Subsample size
    mean_diff = []  # Storage for our subsample mean differences
    for i in range(1000):
        dx = df1.sample(2*m)    # We need two subsamples of size m
        dx1 = dx.iloc[0:m, :]   # First subsample
        dx2 = dx.iloc[m:, :]    # Second subsample
        mean_diff.append(dx1.BPXSY1.mean() - dx2.BPXSY1.mean())  # The difference of mean values
# (sub)sample size, n = 100, 200, 400, 800
# number of samples taken = 1000


In [ ]:
## Inferential statistics
import statsmodels.api as sm


#? z-critical value
stats.norm.ppf(0.05)            # left tail test
stats.norm.ppf(1-0.05)          # right tail test
stats.norm.ppf(1-0.05/2)          # two tail test
# area leftwards of returned critical value

#? t-critical value
stats.t.ppf(q=0.05, df=99)          # left tail test
stats.t.ppf(q=1-0.05, df=99)        # right tail test
stats.t.ppf(q=1-0.025, df=99)         # two tail test
# q = area leftwards of returned critical value



#? p-value (z-statistic)
stats.norm.cdf(-0.77)               # left tail test
1-stats.norm.cdf(1.87)              # right tail test 
stats.norm.cdf(-1.24)*2             # two tail test

stats.norm.sf(abs(-0.77))           # left tail test
stats.norm.sf(abs(1.87))            # right tail test 
stats.norm.sf(abs(1.24))*2          # two tail test

#? p-value (t-statistic)
stats.t.cdf(-0.5,14)                # left tail test
1-stats.t.cdf(1.5,14)               # right tail test 
stats.t.cdf(-0.5,14)*2              # two tail test

stats.t.sf(abs(-0.5), df=14)        # left tail test
stats.t.sf(abs(1.5), df=14)         # right tail test 
stats.t.sf(abs(0.5), df=14)*2       # two tail test



# CI = Best Estimate +/- Margin of Error
# Margin of Error = z*/t* x (estimated) standard error

#? Confidence interval - Population Proportion
# standard error for population proportion = sqrt(P_hat*(1-P_hat)/n)
#* manually
z_star = stats.norm.ppf(0.05/2)     # 95% CI, should be around 1.96 / 2
p = .75
n = 500
se = np.sqrt((p*(1-p))/n)
lcb = p - z_star*se
ucb = p + z_star*se
(lcb, ucb)

#* statsmodels
sm.stats.proportion_confint(n*p, n, alpha=0.05, method='normal')   # Confidence interval for a binomial population proportion
# count: number of success; nobs: total number of trials; alpha: significance level



#? Confidence interval - Difference in Population Proportions
# standard error for difference in two population proportion = sqrt(se1^2 + se2^2 ) = sqrt(P_hat_1*(1-P_hat_1)/n1 + P_hat_2*(1-P_hat_2)/n2)
#* manually
z_star = stats.norm.ppf(0.05/2)
p1 = .304845
n1 = 3000
se_1 = np.sqrt(p1 * (1 - p1)/n1)

p2 = .513258
n2 = 2750
se_2 = np.sqrt(p2 * (1 - p2)/n2)

se_diff = np.sqrt(se_1**2 + se_2**2)

d = p1 - p2
lcb = d - z_star * se_diff
ucb = d + z_star * se_diff
(lcb, ucb)



#? Confidence interval - Mean
# standard error for mean = SD/sqrt(n)
#* manually
t_star = stats.t.ppf(0.05/2, 999)   # 95% CI, df=n-1
mean = df1['column1'].mean()
sd = df1['column1'].std(ddof=1)     # ddof=1 to find sample SD
n = len(df1)
se = sd/np.sqrt(n)      # can also calculate sem using scipy.stats.sem()
lcb = mean - t_star*se
ucb = mean + t_star*se
(lcb, ucb)

#* scipy
stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(n))
stats.t.interval(0.95, loc=mean, scale=sd/np.sqrt(n))

#* statsmodels
sm.stats.DescrStatsW(df1['column1']).zconfint_mean(alpha=0.05, alternative='two-sided')
sm.stats.DescrStatsW(df1['column1']).tconfint_mean(alpha=0.05, alternative='two-sided')    # two-sided confidence interval for weighted mean of data
# For alternative, 'two-sided'-> H1: mean != value, 'larger'-> H1: mean > value, 'smaller'-> H1: mean < value



#? Confidence interval - Difference in Independent Population Means (unequal variance)
# standard error for difference in two population mean = sqrt(se1^2 + se2^2 ) = sqrt(SD1^2/n1+SD2^2/n2)
#* manually
t_star = stats.t.ppf(0.05/2, 498) # df=n1+n2-2
sem_1 = 7.753319 / np.sqrt(3000)            # can also calculate sem using scipy.stats.sem(), 
sem_2 = 6.252568 / np.sqrt(2750)            # by default, np.std uses ddof=0 while stats.sem uses ddof=1
sem_diff = np.sqrt(sem_1**2 + sem_2**2)     # set ddof to 1 for sample SD, ddof to 0 for population SD

d = 29.939946 - 28.778072   # mean_1 - mean_2
lcb = d - t_star * sem_diff
ucb = d + t_star * sem_diff
(lcb, ucb)



#? Confidence interval - Difference in Paired Population Means
# standard error for differences in mean = SDd/sqrt(n)
#* manually
t_star = stats.t.ppf(0.05/2, 999)   # df=n-1
diff = df1['column1'] - df1['column2']
mean_d = diff.mean()
sd_d = diff.std(ddof=1)
n_d = len(diff)
sem_d = sd_d/np.sqrt(n_d)
lcb = mean_d - t_star * sem_d
ucb = mean_d + t_star * sem_d
(lcb, ucb)




# test statistics = (best estimate - hypothesised estimate) / standard error of estiamte

#? Hypothesis testing - Population Proportion
# z-statistic = (P_hat-P_0)/sqrt(P_0*(1-P_0)/n)
z_stat, p_value = sm.stats.proportions_ztest(100, 250, value=0.5, alternative='two-sided', prop_var=0.5)
# count: # success, nobs: # trials, value: hypothesised proportion / P_0
# alternative: ‘two-sided’ (default), ‘less’, ‘greater’; choose based on H1
# prop_var: the P to calculate standard error, should set to P_0 (by default prop_var = count/nobs aka P_hat)



#? Hypothesis testing - Difference In Population Proportions
# z-statistic = (P_hat1-P_hat2)-n)/sqrt(P_hat*(1-P_hat)*(1/n1+1/n2)), where P_hat=(#success1+#success2)/(n1+n2)
#* statsmodels
z_stat, p_value = sm.stats.proportions_ztest([100,375], [250,500], value=0, alternative='two-sided')
# count: [#success1,#success2], nobs: [n1,n2]
# value: hypothesised proportion = 0, # alternative: ‘two-sided’ (default), ‘less’, ‘greater’; choose based on H1



#? Hypothesis testing - Population Mean
# t-statistic = (X_bar-mu)/(SD/sqrt(n))
#* t-test
t_stat, p_value = stats.ttest_1samp(a=df1['column1'], popmean=100, alternative='two-sided')
# popmean: hypothesised mean
# alternative: ‘two-sided’ (default), ‘less’, ‘greater’

#* z-test
z_stat, p_value = sm.stats.ztest(df1['column1'], value=100, alternative='two-sided')
# alternative: ‘two-sided’ (default), ‘larger’, ‘smaller’; choose based on H1



#? Hypothesis testing - Difference in Independent Population Means
# t-statistic (same variance) = ((X_bar_1-X_bar_2)-0)/(sqrt(((n1-1)*SD1^2+(n2-1)*SD2^2)/(n1+n2-2))*sqrt(1/n1+1/n2))
# t-statistic (different variance) = ((X_bar_1-X_bar_2)-0)/sqrt(SD_1^2/n1+SD_2^2/n2)
#* t-test
t_stat, p_value = stats.ttest_ind(a=df1['column1'], b=df1['column2'], equal_var=True, alternative='two-sided')
# H0: mu_a - mu_b = 0

#* z-test
z_stat, p_value = sm.stats.ztest(df1['column1'], df1['column2'], value=0, alternative='two-sided')    # same variance
var_1 = sm.stats.DescrStatsW(df1['column1'])
var_2 = sm.stats.DescrStatsW(df1['column2'])
z_stat, p_value = sm.stats.CompareMeans(var_1, var_2).ztest_ind(usevar='pooled', alternative='two-sided', value=0)      # same variance
z_stat, p_value = sm.stats.CompareMeans(var_1, var_2).ztest_ind(usevar='unequal', alternative='two-sided', value=0)     # different variance
# H0: x1 - x2 = 0
# alternative: ‘two-sided’ (default), ‘larger’, ‘smaller’; choose based on H1



#? Hypothesis testing - Difference in Paired Population Means
# t-statistic = (X_bar_d-0)/(SD_d/sqrt(n))
#* t-test
t_stat, p_value = stats.ttest_rel(a=df1['column1'], b=df1['column2'], alternative='two-sided')
# H0: mu(a-b) = 0

#* z-test
z_stat, p_value = sm.stats.ztest(df1['column1']-df1['column2'], value=0, alternative='two-sided')
# H0: mu_d = 0
# alternative: ‘two-sided’ (default), ‘larger’, ‘smaller’; choose based on H1



#### Descriptive Statistics
<ul>
    <li>Measures of central tendency:</li>
    <ul>
        <li>Mean</li>
        <ul>
            <li>Average value</li>
            <li>For discrete (whole numbers) and continuous data (decimals)</li>
            <li>Mean is susceptible to outliers and skewness. If the mean comes with large spread value, mean may not be representative.</li>
            <li>Not for categorical data</li>
        </ul>
        <li>Median</li>
        <ul>
            <li>Middle value</li>
            <li>Divides the distribution into half. Half of the data points are less than the median and the other half of them are more than the median.</li>
            <li>Less susceptible to outliers and skewness</li>
            <li>Not for categorical data</li>
        </ul>
        <li>Mode</li>
        <ul>
            <li>Most frequently occurring value</li>
            <li>Suitable for discrete, continuous, and categorical data</li>
            <li>In some data, mode may not be a good representation of centrality</li>
        </ul>
    </ul>
    <li>Measures of Spread/Dispersion:</li>
    <ul>
        <li>Range</li>
        <ul>
            <li>Difference between the highest and lowest value in data</li>
        </ul>
        <li>Quartiles</li>
        <ul>
            <li>Divide data into quarters, four equal parts (Q1, Q2, and Q3) with Q2 sitting at the median (2nd quartile is the median)</li>
        </ul>
        <li>Variance</li>
        <ul>
            <li>Measures the width of its spread from center</li>
            <li>Average squared difference between a variable’s value and the mean</li>
            <li>Less variation/risk is preferred</li>
        </ul>
        <li>Standard Deviation</li>
        <ul>
            <li>Square root of variance</li>
        </ul>
    </ul>
    <li>More important task is to make sense out of the data:</li>
    <ul>
        <li>What does the column's mean and standard deviation tells you?</li>
        <li>Why is the mean and standard deviation so close to each other? What could be the reason for this?</li>
    </ul>
    <li>Pandas Codes:</li>
    <ul>
        <li>df.describe( )</li>
        <li>df.mean( )</li>
        <li>df.std( )</li>
        <li>df.var( )</li>
        <li>df['A'].cov(df['A'])</li>
        <li>df.median( )</li>
        <li>df.mode( )</li>
        <li>df.quantile(q=0.5)</li>
        <li>df.count( )</li>
        <li>df.sum( )</li>
        <li>df.min( )</li>
        <li>df.max( )</li>
        <li>df['A'].abs( )</li>
        <li>df['A'].corr(method='pearson')</li>
        <li>df.kurt( )</li>
        <li>df.skew( )</li>
    </ul>
    <li>Numpy/Scipy Codes:</li>
    <ul>
        <li>stats.describe(n1)</li>
        <li>np.mean(n1) <i>or</i> n1.mean( ) </li>
        <li>np.var(n1) <i>or</i> n1.var()</li>
        <li>np.min(n1) <i>or</i> n1.min( )</li>
        <li>np.max(n1) <i>or</i> n1.max( )</li>
    </ul>
    <li>Statsmodels Codes:</li>
    <ul>
        <li>import statsmodels.api as sm</li>
        <li>a = sm.stats.DescrStatsW(df['A'])</li>
        <li>a.mean</li>
        <li>a.std</li>
        <li>a.var</li>
    </ul>
</ul>

In [ ]:
## Predictive Statistics
import statsmodels.formula.api as smf

#? Linear Regression
#* Set up regression model & evaluate it
m = smf.ols("y ~ X1 + C(X2, Treatment('Reference')) + X3*X4", data=df1).fit()
m2 = smf.ols("np.log(Y) ~ np.log(X)", data=df1).fit()
# Wrap categorical variables with C function; Use Treatment to specify reference group
# Infer interaction effects using *

#* Summary table
m.summary()

#* Parameter estimates
m.params
# intercept and coefficients

#* Standard error of regression
np.sqrt(m.mse_resid)

#* RSQ value
m.rsquared


#? Logistic Regression (generalized linear model)
#* Set up regression model & evaluate it
m = smf.glm("y ~ X1 + X2", data=df1, family=sm.families.Binomial()).fit()


#? Marginal Linear Regression (using GEE to fit GLM to estimate intraclass correlation)
#? when observations are possibly correlated within a cluster but uncorrelated across clusters
#* Set up regression model & evaluate it
m = smf.gee("y ~ 1", data=df1, groups='column0', cov_struct=sm.cov_struct.Exchangeable()).fit()
m.cov_struct.summary()


#? Multilevel Models (when there's high potential for outcomes to be grouped together)
for i in ['column1', 'column2', 'column3']:
    m = smf.gee(i + ' ~ 1', groups='column0', cov_struct=sm.cov_struct.Exchangeable(), data=df1).fit()
    print(i, m.cov_struct.summary())

